In [1]:
## Update sklearn to prevent version mismatches:
!pip install sklearn --upgrade

In [2]:
## Install joblib (to save model): 
## *Restart your kernel after installing 
!pip install joblib

In [3]:
## Dependencies
import numpy as np
import pandas as pd

## Read CSV & Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("../a_Resources/Data/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
print(f"Shape: {df.shape}")
df.head(10)

Shape: (6991, 41)


,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
5,CONFIRMED,0,0,0,0,2.566589,1.780000e-05,-1.780000e-05,179.554370,0.004610,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714
6,CONFIRMED,0,0,0,0,16.068647,1.090000e-05,-1.090000e-05,173.621937,0.000517,...,-83,4.485,0.083,-0.028,0.848,0.033,-0.072,286.99948,48.375790,15.841
7,CONFIRMED,0,0,0,0,2.470613,2.700000e-08,-2.700000e-08,122.763305,0.000009,...,-78,4.457,0.024,-0.024,0.964,0.038,-0.038,286.80847,49.316399,11.338
8,CONFIRMED,0,1,0,0,2.204735,4.300000e-08,-4.300000e-08,121.358542,0.000016,...,-89,4.019,0.033,-0.027,1.952,0.099,-0.110,292.24728,47.969521,10.463
9,CONFIRMED,0,0,0,0,3.522498,1.980000e-07,-1.980000e-07,121.119423,0.000047,...,-137,4.169,0.055,-0.045,1.451,0.110,-0.110,281.28812,42.451080,13.563


In [5]:
## DataFrame Exploration
# df.iloc[:,0:10].head(10)

## Set Target & Select Features

In [6]:
target = df['koi_disposition']
target.value_counts()

FALSE POSITIVE    3504
CONFIRMED         1800
CANDIDATE         1687
Name: koi_disposition, dtype: int64

In [7]:
features = df.drop(columns=['koi_disposition'])
print(f"Features: {len(features.columns)}")

Features: 40


## Train / Test Split

In [8]:
## Train / Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

In [9]:
## Check split
print(round(len(y_train)/len(target), 2))
print(round(len(y_test)/len(target), 2))

0.75
0.25


## Pre-processing

In [10]:
## Encode Target
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

## Label encoding
label_encoder = LabelEncoder()
label_encoder.fit(target)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [11]:
## Target names
target_names = label_encoder.inverse_transform([0, 1, 2])
target_names

array(['CANDIDATE', 'CONFIRMED', 'FALSE POSITIVE'], dtype=object)

In [12]:
## Scale Features
from sklearn.preprocessing import MinMaxScaler

## Create MinMaxScaler model & Fit to training data
X_scaler = MinMaxScaler().fit(X_train)

## Transform training & testing data using X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Model Training
### Logistic Regression

In [13]:
## Define model: Logistic Regression
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)

In [14]:
## Fit model
model.fit(X_train_scaled, y_train_encoded)

LogisticRegression(max_iter=1000)

In [15]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train_encoded)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test_encoded)}")

Training Data Score: 0.8481785237459469
Testing Data Score: 0.8443935926773455


In [16]:
predictions = model.predict(X_test_scaled)
print(f"First 10 predictions: {label_encoder.inverse_transform(predictions[:10]).tolist()}")
print(f"First 10 real: {label_encoder.inverse_transform(y_test_encoded[:10]).tolist()}")

First 10 predictions: ['FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CONFIRMED', 'CANDIDATE', 'CANDIDATE', 'FALSE POSITIVE']
First 10 real: ['FALSE POSITIVE', 'CANDIDATE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', 'CONFIRMED', 'CANDIDATE', 'CANDIDATE', 'CANDIDATE', 'FALSE POSITIVE']


In [17]:
## Classification Report
from sklearn.metrics import classification_report
# predictions = model.predict(X_test_scaled)
print(classification_report(y_test_encoded, predictions, target_names=target_names))

                precision    recall  f1-score   support

     CANDIDATE       0.70      0.63      0.67       411
     CONFIRMED       0.72      0.75      0.73       484
FALSE POSITIVE       0.98      1.00      0.99       853

      accuracy                           0.84      1748
     macro avg       0.80      0.79      0.80      1748
  weighted avg       0.84      0.84      0.84      1748



In [18]:
## ROC AUC Score
from sklearn import metrics

y_score = model.predict_proba(X_test_scaled)
print('ROC_AUC_Score: ', metrics.roc_auc_score(y_test_encoded, y_score, multi_class='ovr'))

ROC_AUC_Score:  0.9427362051751453


## Feature Selection

In [22]:
# model.feature_importances_

In [19]:
## Feature Exploration
features.columns

Index(['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
       'koi_period', 'koi_period_err1', 'koi_period_err2', 'koi_time0bk',
       'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1',
       'koi_impact_err2', 'koi_duration', 'koi_duration_err1',
       'koi_duration_err2', 'koi_depth', 'koi_depth_err1', 'koi_depth_err2',
       'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 'koi_teq', 'koi_insol',
       'koi_insol_err1', 'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num',
       'koi_steff', 'koi_steff_err1', 'koi_steff_err2', 'koi_slogg',
       'koi_slogg_err1', 'koi_slogg_err2', 'koi_srad', 'koi_srad_err1',
       'koi_srad_err2', 'ra', 'dec', 'koi_kepmag'],
      dtype='object')

In [20]:
## Select features criteria: Remove error columns
selected_features = df[[
    'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
    'koi_period', 'koi_time0bk', 'koi_impact', 'koi_duration',
    'koi_depth', 'koi_prad', 'koi_teq', 'koi_insol',
    'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_slogg',
    'koi_srad', 'ra', 'dec', 'koi_kepmag'
]]
print(f"Shape: {selected_features.shape}")
selected_features.head(10)

Shape: (6991, 20)


,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_duration,koi_depth,koi_prad,koi_teq,koi_insol,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,162.513840,0.586,4.50700,874.8,2.83,443,9.11,25.8,2,5455,4.467,0.927,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,175.850252,0.969,1.78220,10829.0,14.60,638,39.30,76.3,1,5853,4.544,0.868,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,170.307565,1.276,2.40641,8079.2,33.46,1395,891.96,505.6,1,5805,4.564,0.791,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,171.595550,0.701,1.65450,603.3,2.75,1406,926.16,40.9,1,6031,4.438,1.046,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,172.979370,0.762,3.14020,686.0,2.77,1160,427.65,40.2,2,6046,4.486,0.972,296.28613,48.224670,15.714
5,0,0,0,0,2.566589,179.554370,0.755,2.42900,226.5,1.59,1360,807.74,15.0,3,6046,4.486,0.972,296.28613,48.224670,15.714
6,0,0,0,0,16.068647,173.621937,0.052,3.53470,4914.3,5.76,600,30.75,161.9,1,5031,4.485,0.848,286.99948,48.375790,15.841
7,0,0,0,0,2.470613,122.763305,0.818,1.74319,14231.0,13.04,1339,761.46,4304.3,1,5820,4.457,0.964,286.80847,49.316399,11.338
8,0,1,0,0,2.204735,121.358542,0.224,3.88864,6674.7,16.10,2048,4148.92,5945.9,1,6440,4.019,1.952,292.24728,47.969521,10.463
9,0,0,0,0,3.522498,121.119423,0.631,3.19843,9145.7,14.59,1521,1264.67,1741.5,1,6225,4.169,1.451,281.28812,42.451080,13.563


In [21]:
print(f"Selected Features: {len(selected_features.columns)}")

Selected Features: 20


## Train Selected Features Model

In [23]:
## Train / Test Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(selected_features, target, random_state=42)

## Create MinMaxScaler model & Fit to training data
X_scaler = MinMaxScaler().fit(X_train)

## Transform training & testing data using X_scaler
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Fit model
model.fit(X_train_scaled, y_train_encoded)

LogisticRegression(max_iter=1000)

In [24]:
## Model Train / Test Score
print(f"Training Data Score: {model.score(X_train_scaled, y_train_encoded)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test_encoded)}")

Training Data Score: 0.8199504100705702
Testing Data Score: 0.8009153318077803


In [25]:
## Classification Report
from sklearn.metrics import classification_report
predictions = model.predict(X_test_scaled)
print(classification_report(y_test_encoded, predictions, target_names=target_names))

                precision    recall  f1-score   support

     CANDIDATE       0.60      0.56      0.58       411
     CONFIRMED       0.65      0.66      0.65       484
FALSE POSITIVE       0.98      1.00      0.99       853

      accuracy                           0.80      1748
     macro avg       0.74      0.74      0.74      1748
  weighted avg       0.80      0.80      0.80      1748



In [26]:
## ROC AUC Score
from sklearn import metrics

y_score = model.predict_proba(X_test_scaled)
print('ROC_AUC_Score: ', metrics.roc_auc_score(y_test_encoded, y_score, multi_class='ovr'))

ROC_AUC_Score:  0.9199475802219599


## Model Tuning (Hyperparameters)

In [27]:
## Get list of available parameters
model.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [28]:
## Create GridSearchCV model
from sklearn.model_selection import GridSearchCV

param_grid = {'penalty': ['l1', 'l2'],
              'C': np.logspace(-4, 4, 20),
              'solver': ['liblinear']}

grid = GridSearchCV(model, param_grid, verbose=3)

In [29]:
# param_grid = {'C': [1, 5, 10], 
#               'gamma': [0.0001, 0.001, 0.01]}

In [30]:
# Train the model with GridSearch
# grid.fit(X_train_scaled, y_train_encoded)

In [31]:
# print(grid.best_params_)
# print(grid.best_score_)

## Train Tuned Model

In [32]:
## Tuned model
model = LogisticRegression(C=10000, penalty='l1', solver='liblinear', max_iter=1000)

## Fit model
model.fit(X_train_scaled, y_train_encoded)

LogisticRegression(C=10000, max_iter=1000, penalty='l1', solver='liblinear')

In [33]:
## Model Train / Test Score
print(f"Training Data Score: {model.score(X_train_scaled, y_train_encoded)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test_encoded)}")

Training Data Score: 0.8262445164981881
Testing Data Score: 0.8152173913043478


In [34]:
## Classification Report
from sklearn.metrics import classification_report

predictions = model.predict(X_test_scaled)
print(classification_report(y_test_encoded, predictions, target_names=target_names))

                precision    recall  f1-score   support

     CANDIDATE       0.62      0.61      0.62       411
     CONFIRMED       0.68      0.67      0.67       484
FALSE POSITIVE       0.98      1.00      0.99       853

      accuracy                           0.82      1748
     macro avg       0.76      0.76      0.76      1748
  weighted avg       0.81      0.82      0.81      1748



In [35]:
## ROC AUC Score
from sklearn import metrics

y_score = model.predict_proba(X_test_scaled)
print('ROC_AUC_Score: ', metrics.roc_auc_score(y_test_encoded, y_score, multi_class='ovr'))

ROC_AUC_Score:  0.9277138007368911


## Save Tuned Model

In [36]:
import joblib
filename = 'logistic_regression.sav'
joblib.dump(model, filename)

['logistic_regression.sav']